In [ ]:
# Author: Muhammad Tahir
# Descrition:  This notebook will insert Slicebread FB Costs to Warehouse.

In [ ]:
import pandas as pd
from requests import get,post
import json
import time
from datetime import datetime
import logging
import petl as etl
import pyodbc
import sys 
import pprint

from load_env_var import *
from connection import *
from general_helper import *

In [ ]:
# Remove comment from below line to run in debug mode
logging.basicConfig(level=logging.DEBUG)

# For getting rid of ascii errors
reload(sys)
sys.setdefaultencoding("ISO-8859-1")

In [ ]:
tradbDev_conn = mongodb_connection(tradbDev_str,tradbDev_db)
traDataDump_conn = mongodb_connection(traDataDump_str,traDataDump_db)
print(tradbDev_conn, "\n" ,traDataDump_conn)

In [ ]:
dw_conn = get_dw_conn(server,database,username,password)
cursor = dw_conn.cursor()

In [ ]:
es=elasticsearch_connection(elasticsearch_conn_str)
print(es)

In [ ]:
corp_weeks = json.load(open('corp_weeks.json'))

In [ ]:
PARTICIPANT = 'Slicebread'
REQUIRED_COLUMNS = [['Campaign Name', 'CampaignId', 'adsetid', 'adid', 'Day', 'AdSetName', 'AdName', 
                     'social_reach', 'impressions', 'frequency', 'Result Type', 'Results', 'Cost per Result', 
                     'spent', 'cpc', 'page_likes', 'post_engagement', 'video_views', 'cost_per_lead_lp', 
                     'leads_form', 'clicks','dim_dateid', 'CorpWeekId', 'dim_participantid', 'dimProgramId']]


In [ ]:
ftp = get_ftp_connection(ftp_path,ftp_username,ftp_password)
ftp.cwd('/fbreports/slicedbreadLSR')
files_on_ftp = ftp.nlst()
# Cleaning existing file lists
try:
    files_on_ftp.remove('.')
    files_on_ftp.remove('..')
except:
    pass

In [ ]:
# for filename in files_on_ftp:
    
#     if check_existing_files(PARTICIPANT,filename) == False:
#         # Old file
#         continue
    
#     print('Downloading', filename)
#     ftp.retrbinary('RETR ' + filename, open(filename, 'wb').write)
#     time.sleep(1)
#     fb_costs = pd.read_csv(filename,dtype={'Campaign ID':str,'Ad Set ID':str, 'Ad ID': str}).fillna('')
filename = 'Slicedbread-LSR-Daily-Feb-9-2021.csv'
print(len(fb_costs), 'Number of Rows in file.')

if len(fb_costs) == 0:
    print('0 Rows found in file, Continue to next file.')
    # insert_into_fbCostFile(filename, PARTICIPANT)
    continue

fb_costs = fb_costs.rename(columns={
    'Campaign ID': 'CampaignId', 'Ad Set ID': 'adsetid', 'Ad set ID': 'adsetid',
    'Ad ID': 'adid', 'Ad Set Name': 'AdSetName', 'Ad set name': 'AdSetName',
    'Ad Name': 'AdName', 'Ad name': 'AdName', 'Reach': 'social_reach',
    'Impressions': 'impressions','Frequency': 'frequency',
    'Amount Spent (USD)': 'spent', 'Amount spent (USD)': 'spent',
    'CPC (All)': 'cpc', 'CPC (all)': 'cpc','Page Likes': 'page_likes', 
    'Post Engagement': 'post_engagement', 'Post engagement': 'post_engagement', 'Video Plays': 'video_views',
    'Video plays': 'video_views', 'Cost per Lead': 'cost_per_lead_lp', 'Page likes': 'page_likes', 
    'Leads (Form)': 'leads_form', 'Leads (form)': 'leads_form',
    'Clicks (All)': 'clicks', 'Clicks (all)': 'clicks', 
    'Campaign Name': 'CampaignName','Campaign name': 'CampaignName'})

# Converting date string into datetime object
fb_costs['Day'] = pd.to_datetime(fb_costs['Day'])

# Converting datetime object to Corp Week Id
fb_costs['CorpWeek'] = fb_costs[['Day']].apply(lambda x: corp_weeks.get(str(x[0])[:10], 0), axis=1)

# Attaching Corp Week Id using Dim_CorpWeek table in warehouse

# dw_conn = get_dw_conn()
Dim_CorpWeek = pd.read_sql("""
SELECT * FROM Dim_CorpWeek""", dw_conn)

Dim_Date = pd.read_sql("""
SELECT * from Dim_Date""", dw_conn)

Dim_CorpWeek.CorpWeek = Dim_CorpWeek.CorpWeek.apply(int)
fb_costs = pd.merge(left=fb_costs, right=Dim_CorpWeek, how='left', on='CorpWeek')
fb_costs['date'] = fb_costs[['Day']].apply(lambda x: str(x[0])[:10], axis=1)
Dim_Date.date = Dim_Date.date.apply(str)
fb_costs = pd.merge(left=fb_costs, right=Dim_Date, on='date', how='left')
fb_costs = fb_costs.rename(columns={'DateId': 'dim_dateid','Day': 'dateCreated'})

REQUIRED_COLUMNS = ['CampaignName', 'CampaignId', 'adsetid', 'adid', 'AdSetName', 'AdName',
 'social_reach', 'impressions', 'frequency', 'spent',
 'clicks', 'CorpWeekId', 'dim_dateid','CorpWeek', 'dateCreated','date']
fb_costs = fb_costs[REQUIRED_COLUMNS]

fb_costs['dim_participantid'] = 18050
fb_costs['dimProgramId'] = 1000000195
fb_costs['CampaignManagerId'] = 87
fb_costs['resort'] = 'LSR'

for i in ['CampaignName', 'adsetid', 'adid', 'AdSetName', 'AdName', 'resort']:
    fb_costs[i] = fb_costs[i].fillna('').apply(str)

for i in ['social_reach', 'impressions', 'page_likes', 'post_engagement', 'video_views', 
        'clicks', u'CorpWeekId', 'dim_dateid', 'dim_participantid', 'dimProgramId', 'CampaignManagerId']:
    fb_costs[i] = fb_costs[i].fillna(0).replace('', 0).apply(int)

for i in ['frequency', 'spent', 'cpc', 'post_engagement', 'video_views', 'cost_per_lead_lp']:
    fb_costs[i] = fb_costs[i].fillna(0.0).replace('', 0.0).apply(float)

# decoding ascii's
fb_costs['AdSetName'] = fb_costs[['AdSetName']] \
    .apply(lambda x: encode_agent_name(x[0]).title(), axis=1)

In [ ]:
# Inserting file log into warehous
# insert_into_fbCostFile(filename, PARTICIPANT)

# test = fb_costs.loc[fb_costs.CorpWeekId == 0]

# fb_costs = fb_costs.loc[fb_costs.spent != 0]

# if len(test) > 0:
#     print('Got 0 in CorpWeekId,', filename)

# table = etl.fromdataframe(fb_costs)
# etl.io.appenddb(table, dw_conn, 'Fact_FbCampaignDev')

In [ ]:
# dw_conn.commit()
# dw_conn.close()
# ftp.quit()

In [ ]:
# Service Charges for Slicebread are 25% of ad spent value = 0.25 * spent
charges=0.25

tags_collection=traDataDump_conn["tags"]
adsSpent_collection = tradbDev_conn["spentData"]

elasticIndexBulk=[]
mongodbCollectionBulk=[]

for v in fb_costs.itertuples():
    spentDataPayload={}
    spentDataPayload["contactAttempts"]=getTradbTags(tags_collection,int(v.CampaignManagerId))
    spentDataPayload["campaign"]=v.CampaignId
    spentDataPayload["spent"]= calculate_spent(v.spent,charges)
    spentDataPayload["LeadCorpWeek"]=v.CorpWeek
    spentDataPayload["dateCreated"]=v.date
    spentDataPayload["campaignManagerId"]=v.CampaignManagerId
    spentDataPayload["type"]= "spend"
    spentDataPayload["subtype"]="service_charges" 
    pprint.pprint(spentDataPayload)
    elasticIndexBulk.append(json.dumps(spentDataPayload))
    mongodbCollectionBulk.append(spentDataPayload)
    break

In [ ]:
# Insert in MongoDb and Elasticsearch
mongodb_many_to_many_insert(adsSpent_collection,mongodbCollectionBulk)
adSpentElasticIndexing(es, docs_container,FbAdSpentIndex,FbAdSpentDoctype)

In [ ]:
sum(fb_costs.spent)

In [ ]:
fb_costs.CampaignManagerId.value_counts()

In [ ]:
fb_costs.AdName.value_counts()